# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846831919626                   -0.70    4.5         
  2   -7.852316714790       -2.26       -1.53    1.0   25.0ms
  3   -7.852614044791       -3.53       -2.56    1.5   26.8ms
  4   -7.852645862731       -4.50       -2.87    2.8   34.8ms
  5   -7.852646474388       -6.21       -3.15    1.0   24.9ms
  6   -7.852646679565       -6.69       -4.24    1.0   25.1ms
  7   -7.852646686671       -8.15       -4.90    2.2   33.0ms
  8   -7.852646686722      -10.29       -5.40    1.5   27.1ms
  9   -7.852646686729      -11.17       -5.78    1.2   27.0ms
 10   -7.852646686730      -11.99       -6.51    1.0   25.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846763416077                   -0.70           4.5         
  2   -7.852527484157       -2.24       -1.63   0.80    2.2    275ms
  3   -7.852637149975       -3.96       -2.72   0.80    1.0   23.8ms
  4   -7.852646486631       -5.03       -3.28   0.80    2.2   31.5ms
  5   -7.852646670183       -6.74       -4.14   0.80    1.0   24.2ms
  6   -7.852646686520       -7.79       -4.74   0.80    2.2   32.6ms
  7   -7.852646686720       -9.70       -5.47   0.80    1.5   26.1ms
  8   -7.852646686729      -11.05       -6.54   0.80    1.8   27.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.392458e+01     3.313986e+00
 * time: 0.4027440547943115
     1     1.296880e+00     1.791472e+00
 * time: 0.6005160808563232
     2    -1.710809e+00     2.073823e+00
 * time: 0.6249191761016846
     3    -3.831514e+00     1.709645e+00
 * time: 0.6602730751037598
     4    -5.278277e+00     1.639737e+00
 * time: 0.6945970058441162
     5    -6.923993e+00     8.726145e-01
 * time: 0.7298040390014648
     6    -6.941958e+00     1.133358e+00
 * time: 0.7546670436859131
     7    -7.623218e+00     9.262458e-01
 * time: 0.779818058013916
     8    -7.685034e+00     1.349921e+00
 * time: 0.8045182228088379
     9    -7.743378e+00     3.200082e-01
 * time: 0.8399660587310791
    10    -7.798322e+00     5.038182e-01
 * time: 0.8643290996551514
    11    -7.817333e+00     1.068974e-01
 * time: 0.8885130882263184
    12    -7.827661e+00     7.643262e-02
 * time: 0.9128851890563965
    13    -7.839297e+00     5.461717e-02
 * time: 0.93687915802

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846837983364                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645914524                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.69s
  3   -7.852646686730      -13.30       -7.25    217ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.05526864622207e-7
|ρ_newton - ρ_scfv| = 2.2726550040263757e-7
|ρ_newton - ρ_dm|   = 6.11556233853583e-10
